# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [41]:
import json
import time
import re
import requests
import collections
from typing import List, Counter, Dict, Tuple
import pandas as pd
from urllib.parse import parse_qs, urlencode, urlparse
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer


from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


### Exploratory Data Analysis

In [5]:
## Load the dictionary
def load_dictionary(path: str) -> List[str]:
    """
    Load the dictionary from a file where each line represents a word.
    """
    with open(path, 'r') as file:
        words = [line.strip().lower() for line in file.readlines()]
    return words

In [6]:
## Alphabet
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
path_to_word_dict = './words_250000_train.txt'

# Load the dictionary
words = load_dictionary(path_to_word_dict)

In [7]:
vowels = ['e', 'i', 'a', 'o', 'u'] # vowels ordered by freq
from collections import defaultdict


vowels = ['e', 'i', 'a', 'o', 'u'] # vowels ordered by freq
# max len word in words
max_length = max([len(word) for word in words])
n_word_dictionary = defaultdict(list)

# Optimized substring extraction
for count in range(3, max_length + 1):
    for word in words:
        if len(word) >= count:
            # Use list comprehension or generator for better performance
            substrings = [word[i:i+count] for i in range(len(word)-count+1)]
            n_word_dictionary[count].extend(substrings)

In [ ]:

# Perform preliminary data analysis
def analyze_letter_counts(words: List[str]) -> pd.DataFrame:
    """
    Analyze letter counts and compute the most repeated letters.
    """
    letter_counts = {}
    for word in words:
        for letter in word:
            letter_counts[letter] = letter_counts.get(letter, 0) + 1

    total_letters = sum(letter_counts.values())
    letter_df = pd.DataFrame(
        {
            "Letter": letter_counts.keys(),
            "Count": letter_counts.values(),
            "Proportion": [count / total_letters for count in letter_counts.values()],
        }
    ).sort_values(by="Count", ascending=False)
    return letter_df


def compute_cooccurrence_matrix(words: List[str]) -> pd.DataFrame:
    """
    Compute a normalized co-occurrence matrix of letters appearing together in words.
    """
    # Initialize an empty DataFrame for co-occurrence
    cooccurrence = pd.DataFrame(0, index=alphabet, columns=alphabet)

    # Count co-occurrences of letters
    for word in words:
        unique_letters = set(word)  # Remove duplicates in the word
        for letter1 in unique_letters:
            for letter2 in unique_letters:
                cooccurrence.at[letter1, letter2] += 1

    # Normalize by row sums
    #row_sums = cooccurrence.sum(axis=1)
    #row_sums.replace(0, 1, inplace=True)  # Replace zero sums with 1 to avoid division errors
    #correlation_matrix = cooccurrence.div(row_sums, axis=0)

    return cooccurrence


def plot_letter_distributions(letter_df):
    """
    Plot letter distributions using matplotlib.

    Args:
        letter_df (pd.DataFrame): DataFrame with letter counts and proportions.
    """
    # Sort by count for better visualization
    letter_df_sorted = letter_df.sort_values(by="Count", ascending=False)

    # Bar plot for letter counts
    plt.figure(figsize=(12, 6))
    plt.bar(letter_df_sorted["Letter"], letter_df_sorted["Count"])
    plt.title("Letter Frequency Distribution")
    plt.xlabel("Letters")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Bar plot for letter proportions
    plt.figure(figsize=(12, 6))
    plt.bar(letter_df_sorted["Letter"], letter_df_sorted["Proportion"])
    plt.title("Letter Proportion Distribution")
    plt.xlabel("Letters")
    plt.ylabel("Proportion")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:

def generate_ngrams_dict(words: List[str], n: int = 2) -> Dict:
    """
    Generate n-grams from a list of words.
    """
    ngram_counts = {}
    for word in words:
        for i in range(len(word) - n + 1):
            ngram = word[i:i + n]
            ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1
    return ngram_counts

# Normalize n-gram counts to probabilities
def compute_ngram_probabilities(ngram_counts: Counter) -> Dict[str, float]:
    """
    Compute probabilities from n-gram counts.
    """
    total_ngrams = sum(ngram_counts.values())
    return {ngram: count / total_ngrams for ngram, count in ngram_counts.items()}



In [ ]:
# Generate n-grams using a normal dictionary
bigrams_dict = generate_ngrams_dict(words, n=2)
prob = compute_ngram_probabilities(bigrams_dict)

# Analyze letter counts
letter_analysis = analyze_letter_counts(words)
# Plot letter distributions
plot_letter_distributions(letter_analysis)

In [ ]:
# Compute coocurrance matrix
#letter_cooccurrence = compute_cooccurrence_matrix(words)
#plt.figure(figsize=(12, 12))
#plt.imshow(letter_cooccurrence, cmap='coolwarm')
#plt.colorbar()
#plt.title("Letter Concurrence Matrix")
#plt.xticks(range(len(alphabet)), alphabet)
#plt.yticks(range(len(alphabet)), alphabet)
# remove grid
#plt.grid(False)
# display value in each cell in little with only ".x" value and small font
#for i in range(len(alphabet)):
#    for j in range(len(alphabet)):
#        plt.text(j, i, f"{letter_cooccurrence.iloc[i, j]:.01f}", ha='center', va='center', color='black', fontsize=8)
#plt.show()

##### Bigram, Trigram Analysis
# Function to get most common n-grams
def most_common_ngrams(ngrams_dict: Dict[str, int], top_n: int = 20) -> List[Tuple[str, int]]:
    """
    Get the most common n-grams.

    Args:
        ngrams_dict (Dict[str, int]): Dictionary of n-grams and their counts.
        top_n (int): Number of top n-grams to return.

    Returns:
        List[Tuple[str, int]]: Sorted list of n-grams and their counts.
    """
    sorted_ngrams = sorted(ngrams_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_ngrams[:top_n]


# Function to plot most common n-grams
def plot_ngrams(ngrams: List[Tuple[str, int]], title: str):
    """
    Plot the most common n-grams.

    Args:
        ngrams (List[Tuple[str, int]]): List of n-grams and their counts.
        title (str): Title for the plot.
    """
    labels, counts = zip(*ngrams)

    plt.figure(figsize=(12, 6))
    plt.bar(labels, counts)
    plt.title(title)
    plt.xlabel("N-Grams")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


# Generate N-Grams (Bigrams and Trigrams)
trigrams_dict = generate_ngrams_dict(words, n=3)

# Get and Plot Most Common N-Grams
most_common_bigrams = most_common_ngrams(bigrams_dict, top_n=10)
most_common_trigrams = most_common_ngrams(trigrams_dict, top_n=10)

plot_ngrams(most_common_bigrams, "Most Common Bigrams")
plot_ngrams(most_common_trigrams, "Most Common Trigrams")


In [73]:
### Average amount of vowels and consonants 
def calculate_vowel_consonant_ratios(words: List[str], vowels: List[str]) -> Dict[int, float]:
    """
    Calculate the average vowel ratio (vowels/word_length) for each word length.

    Args:
        words (List[str]): List of words from the dictionary.
        vowels (List[str]): List of vowel letters.

    Returns:
        Dict[int, float]: Mapping of word length to average vowel ratio.
    """
    vowel_ratios = {}
    length_counts = {}

    for word in words:
        word_length = len(word)
        if word_length not in vowel_ratios:
            vowel_ratios[word_length] = 0
            length_counts[word_length] = 0

        word_vowels = sum(1 for letter in word if letter in vowels)
        vowel_ratios[word_length] += word_vowels
        length_counts[word_length] += 1

    # Compute average vowel ratio for each word length
    for length in vowel_ratios:
        vowel_ratios[length] /= length_counts[length]

    return vowel_ratios


print(calculate_vowel_consonant_ratios(words, list('aeiou')))

{3: 0.8518855065879146, 6: 2.3428176654214217, 4: 1.4808019670890864, 5: 1.9128969309916621, 8: 3.0884014186260345, 7: 2.7040619700940343, 10: 3.8893258635402366, 9: 3.482560020707953, 11: 4.281751952953568, 12: 4.690174936736715, 13: 5.102114850262427, 15: 5.91402801765496, 14: 5.498622273249139, 20: 7.888888888888889, 17: 6.690140845070423, 16: 6.2758510976773785, 2: 0.4318181818181818, 21: 8.061224489795919, 18: 7.089639115250291, 19: 7.45124716553288, 25: 9.666666666666666, 22: 8.545454545454545, 1: 0.17647058823529413, 23: 9.428571428571429, 29: 9.5, 24: 9.222222222222221, 28: 8.0, 27: 9.0}


### API Code

In [65]:
class HangmanAPI_Combined(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        full_dictionary_path = './words_250000_train.txt'
        model_path = './hangman-model'
        alpha = 0.4  # Weight for combining BERT and statistical distributions
        self.beta = 0.1  # Weight for n-gram influence on the statistical distribution
    
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
    
        self.full_dictionary = self.load_dictionary(full_dictionary_path)
        self.current_dictionary = self.full_dictionary
        self.full_dictionary_common_letter_sorted = collections.Counter(
            "".join(self.full_dictionary)
        ).most_common()
    
        self.model = AutoModelForMaskedLM.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model.eval()
    
        # Initialize API state
        self.guessed_letters = []
        self.alphabet = list("abcdefghijklmnopqrstuvwxyz")
        self.letter_ids = [self.tokenizer.convert_tokens_to_ids(l) for l in self.alphabet]
        self.letter_id_map = {lid: l for l, lid in zip(self.alphabet, self.letter_ids)}
    
        self.alpha = alpha # weight for combining BERT and statistical distributions
        self.vowels = set("aeiou")
    
    @staticmethod
    def load_dictionary(path: str) -> List[str]:

        with open(path, "r") as f:
            return [line.strip() for line in f if line.strip()]


    def filter_current_dictionary_strict(self, clean_word: str) -> List[str]:
        # Filter dict based on exact regex match
        regex_pattern = "^" + clean_word + "$"
        pattern = re.compile(regex_pattern)
        return [w for w in self.current_dictionary if len(w) == len(clean_word) and pattern.match(w)]

        
    def partial_pattern_score(self, partial_word: str, candidate_word: str) -> float:
        """
        Partial matching function
        Scoring rules (can be adjusted):
        - +2 points for each correctly matched known letter (partial_word[i] != '.' and candidate_word[i] == that letter).
        - -1 point if a known letter does not match.
        - 0 points for '.' since it's unknown.
        
        You can add more sophisticated scoring if desired.
        """
        score = 0.0
        for p_char, c_char in zip(partial_word, candidate_word):
            if p_char == '.': # no add
                continue
            else:
                # Known letter
                if p_char == c_char:
                    score += 2.0
                else:
                    score -= 1.0
        return score
    
    
    def filter_current_dictionary_partial(self, clean_word: str, top_n: int = 50) -> List[str]:
        scored_candidates = []
        for w in self.current_dictionary:
            if len(w) == len(clean_word):
                s = self.partial_pattern_score(clean_word, w)
                scored_candidates.append((w, s))
        
        scored_candidates.sort(key=lambda x: x[1], reverse=True)
        return [w for w, s in scored_candidates[:top_n]] if scored_candidates else self.current_dictionary
    
    def filter_current_dictionary(self, clean_word: str, partial_threshold: int = 15):
        """
        Strict filtering + partial filtering
        Fallback to global freq if empty.
        # To improve: optimize thresholds
        """
        strict_matches = self.filter_current_dictionary_strict(clean_word)
        if len(strict_matches) == 0: # No strict matches: use partial pattern ranking to get top candidates
            new_dict = self.filter_current_dictionary_partial(clean_word, top_n=50)
            if len(new_dict) == 0: #
                new_dict = self.full_dictionary  # final fallback
            self.current_dictionary = new_dict
        elif len(strict_matches) < partial_threshold:
            # Few strict matches: combine strict matches with top partial matches to broaden candidate set
            partial_matches = self.filter_current_dictionary_partial(clean_word, top_n=50)
            merged_set = list(set(strict_matches + partial_matches))
            self.current_dictionary = merged_set
        else:
            self.current_dictionary = strict_matches  # Strict matches only

    def compute_ngram_statistics(self, candidate_words: List[str], n: int = 2) -> collections.Counter:
        """
        Compute n-gram frequencies for candidate words.
        """
        ngram_counts = collections.Counter()
        for w in candidate_words:
            for i in range(len(w) - n + 1):
                ngram = w[i:i+n]
                ngram_counts[ngram] += 1
        return ngram_counts

    def compute_statistical_distribution(self, candidate_words: List[str]) -> Dict[str, float]:
        """
        Compute statistical letter frequencies from candidate words.
        """
        c = collections.Counter()
        for word in candidate_words:
            unique_letters = set(word)  # Count presence of each letter in each word
            for letter in unique_letters:
                c[letter] += 1

        # Normalize to probabilities
        total = sum(c.values())
        if total > 0:
            return {letter: count / total for letter, count in c.items()}
        else:
            return {}

    def compute_bert_distribution(self, partial_word: str) -> Dict[str, float]:
        """
        Compute letter probabilities using the BERT model.
        """
        masked_positions = [i for i, ch in enumerate(partial_word) if ch == "."]
        if not masked_positions:
            # No unknown positions
            return {}

        letter_scores = {letter: 0.0 for letter in self.alphabet}
        with torch.no_grad():
            for mask_pos in masked_positions:
                input_tokens = [
                    "[MASK]" if ch == "." and i == mask_pos else ch
                    for i, ch in enumerate(partial_word)
                ]
                masked_input_str = " ".join(input_tokens)
                inputs = self.tokenizer(masked_input_str, return_tensors="pt", truncation=True)
                input_ids = inputs["input_ids"]

                outputs = self.model(**inputs)
                predictions = torch.softmax(outputs.logits, dim=-1)
                mask_index = (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[1].item()

                letter_probs = predictions[0, mask_index]
                for letter, token_id in zip(self.alphabet, self.letter_ids):
                    letter_scores[letter] += letter_probs[token_id].item()

        # Normalize
        total_score = sum(letter_scores.values())
        if total_score > 0:
            return {letter: score / total_score for letter, score in letter_scores.items()}
        else:
            return {}

        
    
    def apply_ngram_weighting(self, stat_dist: Dict[str, float], candidate_words: List[str], clean_word: str, n=2) -> Dict[str, float]:
        """
        Adjust letter distribution using n-gram statistics, scaled by self.beta.
        """
        ngram_counts = self.compute_ngram_statistics(candidate_words, n)
        total_ngrams = sum(ngram_counts.values())
        if total_ngrams == 0:
            return stat_dist
    
        known_chars = [ch for ch in clean_word if ch.isalpha()]
    
        # Collect letters that appear in n-grams containing known chars
        letter_boost = collections.Counter()
        for ngram, count in ngram_counts.items():
            if any(kc in ngram for kc in known_chars):
                for l in set(ngram):
                    if l.isalpha():
                        letter_boost[l] += count
    
        if letter_boost:
            max_boost = max(letter_boost.values())
            for ltr, val in letter_boost.items():
                stat_dist[ltr] = stat_dist.get(ltr, 0) * (1 + self.beta * (val / max_boost))
    
        return stat_dist

    
    def guess(self, partial_word: str) -> str:
        """
        Guess the next letter for the given partial word.
    
        - If more than 3 unknown letters remain, rely solely on the statistical model.
        - If 3 or fewer unknown letters remain, combine statistical and BERT distributions.
        """
        # Convert "_ p p _ e " to ".pp.e"
        clean_word = partial_word.replace(" ", "").replace("_", ".")
    
        # Filter the dictionary based on the partial word
        self.filter_current_dictionary(clean_word)
    
        # Compute statistical distribution
        stat_dist = self.compute_statistical_distribution(self.current_dictionary)
        # Apply n-gram weighting
        stat_dist = self.apply_ngram_weighting(stat_dist, self.current_dictionary, clean_word, n=2)
    
        # Count unknown letters
        unknown_count = clean_word.count('.')
    
        # If unknown_count <= 3, combine with BERT; otherwise, purely statistical
        if unknown_count <= 3:
            bert_dist = self.compute_bert_distribution(clean_word)
        else:
            # No BERT if more than 3 unknowns
            bert_dist = {}
    
        combined_scores = {}
        for letter in self.alphabet:
            if letter not in self.guessed_letters:
                stat_val = stat_dist.get(letter, 0.0)
                bert_val = bert_dist.get(letter, 0.0)
    
                if unknown_count <= 3:
                    # Use combined distribution
                    score = self.alpha * bert_val + (1 - self.alpha) * stat_val
                else:
                    # Use only statistical distribution
                    score = stat_val

                combined_scores[letter] = score
    
        # Pick the best guess
        if combined_scores:
            guess_letter = max(combined_scores, key=combined_scores.get)
        else:
            # Fallback to global frequency if no candidates
            guess_letter = next(
                (l for l, _ in self.full_dictionary_common_letter_sorted if l not in self.guessed_letters), None
            )
    
        if guess_letter:
            self.guessed_letters.append(guess_letter)
    
        return guess_letter
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.filtered_dict = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [66]:
api = HangmanAPI_Combined(access_token="ad5506c7195dcbf44946272eeb68a1", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [79]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 094a2de5db8c. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ _ _ _ _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ _ _ _ n _ _ _ '}
Guessing letter: e
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ i _ _ e _ n e _ _ '}
Guessing letter: s
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ i _ _ e _ n e s s '}
Guessing letter: d
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ e _ _ i _ _ e _ n e s s '}
Guessing letter: t
Sever response: {'game_id': '094a2de5db8c', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ e _ _ i t t e _ n e s s '}
Guessing letter: a
Se

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

### BERT Training script

In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from typing import List, Counter, Dict, Tuple
import pandas as pd
from urllib.parse import parse_qs, urlencode, urlparse
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import numpy as np
from sklearn.model_selection import train_test_split
import os

from urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#######################################
# Set Random Seeds for Reproducibility
#######################################
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

def load_dictionary(path: str) -> List[str]:
    with open(path, "r") as f:
        words = [line.strip() for line in f]
    return words

path_to_word_dict = 'words_250000_train.txt'
words = load_dictionary(path_to_word_dict)
print(calculate_vowel_ratios(words, list('aeiou')))

#######################################
# Data Preparation Functions
#######################################

def prepare_hid_masked_dataset(
    full_dictionary_location: str,
    tokenizer: AutoTokenizer,
    diversity_factor: int = 5,
    late_game_prob: float = 0.3
) -> List[Tuple[str, int]]:
    """
    Prepare a dataset with `[HID]` and `[MASK]` tokens for fine-tuning.
    """
    vowels = set('aeiou')

    # Define probabilities for choosing how many consonants to hide in late-game scenarios
    # Make sure these sum to 1.
    hid_count_options = [0, 1, 2, 3]
    hid_count_probs = [0.1, 0.3, 0.4, 0.2]  # Example distribution

    with open(full_dictionary_location, "r") as f:
        words = [line.strip() for line in f]

    data = []
    for word in words:
        word_tokens = list(word)
        word_length = len(word_tokens)
        if word_length == 0:
            continue

        for _ in range(diversity_factor):
            # Randomly select one position for `[MASK]`
            mask_idx = random.randint(0, word_length - 1)
            masked_word = word_tokens[:]
            masked_word[mask_idx] = "[MASK]"

            target_letter = word_tokens[mask_idx]

            # Decide if this is a late-game scenario
            if random.random() < late_game_prob:
                # Late-game scenario:
                # Select how many consonants to hide according to the given probabilities
                chosen_hid_count = random.choices(hid_count_options, weights=hid_count_probs, k=1)[0]

                # Identify consonant positions (excluding the masked one)
                consonant_positions = [
                    i for i, ch in enumerate(word_tokens)
                    if i != mask_idx and ch.isalpha() and ch not in vowels
                ]

                # If not enough consonants are available, reduce the hid count
                chosen_hid_count = min(chosen_hid_count, len(consonant_positions))

                if chosen_hid_count > 0:
                    hid_indices = random.sample(consonant_positions, chosen_hid_count)
                    for hid_idx in hid_indices:
                        masked_word[hid_idx] = "[HID]"

                # Note: If chosen_hid_count = 0, we don't hide any additional consonants.
            else:
                # Early or mid-game scenario (the original random approach)
                hid_indices = [
                    i for i in range(word_length)
                    if i != mask_idx and random.random() < 0.35
                ]
                for hid_idx in hid_indices:
                    masked_word[hid_idx] = "[HID]"

            # Convert target letter to token ID
            target_token_id = tokenizer.convert_tokens_to_ids(target_letter)
            if target_token_id is None or target_token_id == tokenizer.unk_token_id:
                # If we cannot map the target letter to a token, skip this example
                continue

            # Join tokens with spaces
            masked_input = " ".join(masked_word)
            data.append((masked_input, target_token_id))

    return data

def prepare_late_game_dataset(
    full_dictionary_location: str,
    tokenizer: AutoTokenizer,
    diversity_factor: int = 5,
    max_masks: int = 1
) -> List[Tuple[str, List[int]]]:
    """
    Prepare a dataset that simulates late-game hangman scenarios.
    Each example will have 1 or 2 letters masked (randomly chosen),
    and all other letters are visible or minimally hidden.
    """
    with open(full_dictionary_location, "r") as f:
        words = [line.strip() for line in f if line.strip()]

    data = []
    vowels = set('aeiou')  # if needed for heuristics
    for word in words:
        word_tokens = list(word)
        word_length = len(word_tokens)
        if word_length == 0:
            continue

        for _ in range(diversity_factor):
            # Decide how many letters to mask: either 1 or 2
            # You can weight these probabilities if desired
            num_masks = random.choice([1, 2]) if max_masks > 1 else 1
            if num_masks > word_length:
                # If word too short, just mask 1 letter
                num_masks = 1

            # Randomly choose positions for masks
            mask_positions = random.sample(range(word_length), num_masks)

            masked_word = word_tokens[:]
            target_letters = [word_tokens[pos] for pos in mask_positions]

            # Replace chosen letters with [MASK]
            for pos in mask_positions:
                masked_word[pos] = "[MASK]"

            # Optionally hide a few other letters minimally (late-game often means fewer unknowns)
            # We'll do a light hid: about 10% chance on other letters
            # This is optional and can be tweaked
            for i in range(word_length):
                if i not in mask_positions and random.random() < 0.1:
                    masked_word[i] = "[HID]"

            # Convert target letters to token IDs
            target_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in target_letters]
            if any(t_id is None or t_id == tokenizer.unk_token_id for t_id in target_token_ids):
                # If we can't map all target letters, skip this example
                continue

            masked_input = " ".join(masked_word)
            data.append((masked_input, target_token_ids))

    return data

#######################################
# Custom Dataset
#######################################
class HangmanDataset_Single(Dataset):
    def __init__(self, data: List[Tuple[str, int]], tokenizer: AutoTokenizer, max_length: int = 64):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        masked_input, target_token_id = self.data[idx]

        # Tokenize input
        encoded = self.tokenizer(
            masked_input,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
        )

        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()

        # Create labels
        labels = input_ids.clone()

        # Replace all tokens except `[MASK]` with -100
        labels[labels != self.tokenizer.mask_token_id] = -100

        # Assign target token ID to the `[MASK]`
        mask_indices = (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
        if len(mask_indices) != 1:
            # We expect exactly one mask token per training example
            raise ValueError("Each example must have exactly one [MASK] token.")
        labels[mask_indices[0]] = target_token_id

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

class HangmanDataset_Multi(Dataset):
    def __init__(self, data: List[Tuple[str, List[int]]], tokenizer: AutoTokenizer, max_length: int = 64):
        """
        Custom Dataset for Hangman fine-tuning.

        Args:
            data (List[Tuple[str, List[int]]]): Input-output pairs (masked_word, target_token_ids).
            tokenizer (AutoTokenizer): Tokenizer with `[HID]` and `[MASK]` tokens added.
            max_length (int): Maximum sequence length.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        masked_input, target_token_ids = self.data[idx]

        # Tokenize input with the custom tokenizer
        encoded = self.tokenizer(
            masked_input,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
        )

        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()

        # Create labels
        labels = input_ids.clone()

        # Replace all tokens except `[MASK]` with -100 (ignored in loss calculation)
        labels[:] = -100  # Initialize all tokens to -100

        # Find all `[MASK]` positions and assign corresponding target token IDs
        mask_indices = (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
        if len(mask_indices) != len(target_token_ids):
            raise ValueError("Mismatch between number of [MASK] tokens and target IDs.")

        for mask_idx, target_token_id in zip(mask_indices, target_token_ids):
            labels[mask_idx] = target_token_id

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }


#######################################
# Prepare Tokenizer and Add Tokens
#######################################

new_tokens = ["[HID]", "[MASK]"]
alphabet_tokens = list('abcdefghijklmnopqrstuvwxyz')
# add [HID], [MASK], and all lowercase letters as tokens
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.add_tokens(new_tokens, special_tokens=True)
tokenizer.add_tokens(alphabet_tokens, special_tokens=True)

#######################################
# Prepare the Dataset

#masked_data = prepare_hid_masked_dataset("./words_250000_train.txt", tokenizer, diversity_factor=7)
masked_data = prepare_late_game_dataset("./words_250000_train.txt", tokenizer, diversity_factor=5)
# Split into train and eval sets (5% eval)
train_data, eval_data = train_test_split(masked_data, test_size=0.05, random_state=42)

train_dataset = HangmanDataset_Multi(train_data, tokenizer, max_length=64)
eval_dataset = HangmanDataset_Multi(eval_data, tokenizer, max_length=64)

# Check an example
example = train_dataset[0]
print(f"Input IDs: {example['input_ids']}")
print(f"Attention Mask: {example['attention_mask']}")
print(f"Labels: {example['labels']}")

#######################################
# Model Preparation
#######################################
os.environ["WANDB_DISABLED"] = "true"
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="./hangman-model",
    per_device_train_batch_size=512,
    num_train_epochs=10,
    save_steps=2000,
    evaluation_strategy="steps",  # Evaluate regularly
    eval_steps=2000,  # Evaluate every 1000 steps
    logging_dir="./logs",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


# Resume training from the given checkpoint
trainer.train(resume_from_checkpoint="./hangman-model/checkpoint-21090")
#trainer.train()

# Save the fine-tuned model
model.save_pretrained("./hangman-model")
tokenizer.save_pretrained("./hangman-model")